In [1]:
search_for = 131
start_from = 0
threads = 10
thread = 7
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.06292343139648438
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 296765305
type: [1, 1, 1, 1, 131] 131
cases of this type: 294499921
100000 7.013092214677136
200000 11.310176481064078
300000 11.257564702878454
400000 11.66373858397642
500000 11.040887182479114
600000 13.919522062033888
700000 15.220873659647761
800000 17.783826833486245
900000 20.707026764961185
1000000 21.856339202622053
1100000 18.59564534410068
1200000 9.871787758614865
1300000 21.426491644614934
1400000 21.41402522169902
1500000 21.268115346692117
1600000 21.507944800253092
1700000 22.40338840804786
1800000 22.199910710947783
1900000 21.39247282542509
2000000 21.6581056526532
2100000 21.954896809548273
2200000 21.79852168861446
2300000 5.848289843862875
2400000 23.68391503391654
2500000 27.387536904664813
2600000 26.110898115372805
2700000 26.368576824049143
2800000 26.704298304444436
2900000 25.872834515983275
3000000 25.3

29400000 21.91548100255866
29500000 7.898045369906276
29600000 22.815985667256083
29700000 21.249638698297908
29800000 21.65877371832785
29900000 21.576146824711376
30000000 20.80302606766686
30100000 21.189786298703357
30200000 19.599943408519042
30300000 20.468562009814626
30400000 5.811472511629432
30500000 19.97458341958297
30600000 20.49436961164518
30700000 20.21625623200054
30800000 22.13450063929995
30900000 20.523214265008775
31000000 21.483320074480428
31100000 21.600518958244976
31200000 21.246594798033655
31300000 9.295774404093564
31400000 21.596015608186807
31500000 22.13975011695077
31600000 22.25871819053896
31700000 19.11955400454864
31800000 12.268166479624158
31900000 21.29930028294907
32000000 23.24575554489487
32100000 20.817047254993053
32200000 21.44201957739043
32300000 20.764990268907738
32400000 20.698538560906506
32500000 20.303059634432792
32600000 12.214260848008182
32700000 14.509917246174968
32800000 19.581059701060514
32900000 21.142088229103145
33000000

59200000 17.272528638380617
59300000 17.67785519561039
59400000 17.408534811429153
59500000 16.850168661143115
59600000 7.425444500239015
59700000 15.294169535634783
59800000 17.893000932661476
59900000 17.014931482144277
60000000 17.713507800974725
60100000 18.197109496965556
60200000 18.36206822700269
60300000 7.869954299803115
60400000 18.830066734377144
60500000 20.206529364921177
60600000 19.412402534965405
60700000 19.085372394999286
60800000 19.96295193644749
60900000 18.303090098692934
61000000 19.82011912379211
61100000 17.905972946625837
61200000 9.470202259585346
61300000 17.417303882732142
61400000 17.155690619744394
61500000 17.79889367752739
61600000 16.5673446457989
61700000 17.152670994276093
61800000 13.899197068475322
61900000 8.513666155268131
62000000 16.99080642350373
62100000 15.997135034673482
62200000 17.052529146949688
62300000 16.841102906037207
62400000 17.7841007159223
62500000 9.560310833265763
62600000 15.918356769039264
62700000 19.361545076381777
6280000

89000000 14.89026361371541
89100000 14.594564199543633
89200000 14.499117503418663
89300000 6.632876076477945
89400000 15.12980550564608
89500000 15.005316883145252
89600000 15.375768128723404
89700000 15.62384915514838
89800000 16.03538815454176
89900000 16.17683640817558
90000000 16.550333442419888
90100000 15.915580321850676
90200000 16.34952471143751
90300000 15.892323076951781
90400000 15.802596614743164
90500000 14.638037562267378
90600000 12.42919575912056
90700000 8.622558360221268
90800000 14.30117471285319
90900000 14.4187101460613
91000000 14.82388970278901
91100000 4.172648820032245
91200000 15.227148847051733
91300000 14.787428055253635
91400000 14.501477173252981
91500000 8.22653267512881
91600000 13.30964548407492
91700000 14.468522784979525
91800000 15.671514710218501
91900000 15.889145393249155
92000000 15.947647663586931
92100000 15.960340990376292
92200000 16.031949372595967
92300000 15.983312649371314
92400000 16.066105335312535
92500000 15.518769794737727
92600000 

118000000 10.351114296423683
118100000 5.599992760222664
118200000 11.888509899912878
118300000 5.417986584807049
118400000 12.627468205524506
118500000 12.600436554067558
118600000 12.619281846763908
118700000 12.554066808544796
118800000 12.514448632340487
118900000 12.57608793673723
119000000 13.698611138945175
119100000 13.953085421405152
119200000 14.15017364326207
119300000 14.100141727662864
119400000 13.27842245124479
119500000 13.014294406385364
119600000 12.441800390869002
119700000 13.259182885941659
119800000 12.328228231838008
119900000 12.697393891950995
120000000 12.299274428004317
120100000 5.730661098172038
120200000 12.52699164109907
120300000 5.078216639918199
120400000 10.972637484008551
120500000 6.7798461311215235
120600000 11.133419349196588
120700000 12.798461529024173
120800000 12.301159437088472
120900000 12.85717193881101
121000000 12.46217390897264
121100000 12.832216206708788
121200000 13.266692797454972
121300000 13.64804428926158
121400000 14.092382273732

...

175700000 8.541544610161264
175800000 8.903836371161802
175900000 8.386036919816384
176000000 8.746096083306034
176100000 8.416142632653614
176200000 8.57063463240546
176300000 3.7454318709160983
176400000 8.333560009686298
176500000 3.615484649015754
176600000 7.349899301285981
176700000 5.9350490953109025
176800000 6.189248942312856
176900000 8.512259284660452
177000000 8.772817278094642
177100000 8.210707084210872
177200000 8.808420739475023
177300000 8.50709322613963
177400000 8.7133117563411
177500000 9.445124608144996
177600000 9.074192466798355
177700000 9.652453865262666
177800000 9.1820248684472
177900000 9.044898048526195
178000000 8.45708068221125
178100000 8.498738579669068
178200000 8.61773858807443
178300000 8.028936674031396
178400000 8.44884938579211
178500000 5.172112181047261
178600000 7.307338988933196
178700000 6.709225320917763
178800000 4.02308034748398
178900000 8.256284511657922
179000000 3.5915227398686507
179100000 8.221952481467337
179200000 8.351217690485466

205000000 6.949022628832072
205100000 6.633064683312939
205200000 6.665982994551242
205300000 2.796405948983888
205400000 6.553726612257013
205500000 6.439720569286099
205600000 6.368624066634636
205700000 4.300168682880832
205800000 4.494214506608625
205900000 6.292440200702995
206000000 6.558623543693619
206100000 6.42999242028794
206200000 2.4564696044817667
206300000 6.511902007738601
206400000 6.605997339128617
206500000 6.828793056701492
206600000 7.053098820337188
206700000 7.225060116465297
206800000 7.235711583073378
206900000 7.1727588465213605
207000000 6.980821271394484
207100000 6.984622008628915
207200000 6.808186206126663
207300000 6.608872225109369
207400000 6.44341470442299
207500000 3.9337860303703014
207600000 5.35414745163793
207700000 6.301392160507947
207800000 6.0839072033391535
207900000 6.60233819548941
208000000 1.8366242983469268
208100000 6.618882534873035
208200000 6.167457034712315
208300000 6.212879457517415
208400000 5.019798556154112
208500000 3.8835449

234200000 5.137382775939775
234300000 1.9523146449358595
234400000 5.096744290840953
234500000 4.997454737448841
234600000 4.54641778104291
234700000 4.787750683653831
234800000 4.706434197812892
234900000 4.060130781958848
235000000 2.1495310527035496
235100000 4.515871476184874
235200000 4.777175058122807
235300000 4.602340483055492
235400000 4.769869440509011
235500000 5.004993684404827
235600000 3.2567008121864203
235700000 3.879608648426592
235800000 5.114669595733433
235900000 5.276569975341837
236000000 5.174806159574619
236100000 5.034225028375424
236200000 5.101378845246513
236300000 4.828485541384574
236400000 5.248864622572987
236500000 2.914458051638782
236600000 3.9158654782961633
236700000 4.634618137365368
236800000 4.357247831145048
236900000 4.538044949476238
237000000 4.2873453316977095
237100000 4.248521313366552
237200000 1.819217004469312
237300000 3.8750025182611485
237400000 4.387843728642136
237500000 4.250886285160713
237600000 4.676328385336928
237700000 4.580

263100000 2.9245306440387036
263200000 2.640990800047477
263300000 1.1055417153015834
263400000 2.841967291051574
263500000 2.800142801800161
263600000 2.649852381515748
263700000 2.6790444592159024
263800000 2.674146446041067
263900000 2.405436953667207
264000000 2.660720908291949
264100000 2.3502440282008448
264200000 0.6931098607614836
264300000 2.628823956618637
264400000 2.33492142218104
264500000 2.7189461964834067
264600000 2.428320979688807
264700000 2.8497037377032837
264800000 2.424444667518407
264900000 2.772901024024738
265000000 2.7046881121756927
265100000 0.9677983192980886
265200000 2.7793771334712933
265300000 2.5122297469829618
265400000 2.6903808844747314
265500000 1.520105571247826
265600000 2.101864614812315
265700000 2.4926517006649047
265800000 2.6090939884979925
265900000 2.3960561901789306
266000000 2.517976405642569
266100000 2.3376447175802455
266200000 2.4709682889451683
266300000 2.1607126546660593
266400000 1.4160306930683122
266500000 1.7297786259804169
2

291700000 0.40243925714013973
291800000 0.40815661434651085
291900000 0.34101921857687173
292000000 0.35315893115143976
292100000 0.3232596764792369
292200000 0.320235576348669
292300000 0.07852858711937069
292400000 0.25654410754887597
292500000 0.27430540716786517
292600000 0.2757391664413413
292700000 0.2733021173577971
292800000 0.22656080457807581
292900000 0.20452892625190813
293000000 0.19859664529049395
293100000 0.18820773217215803
293200000 0.2205788305654824
293300000 0.21460515255984997
293400000 0.09304983851678834
293500000 0.1742570520269374
293600000 0.16347491065734956
293700000 0.1420605963536501
293800000 0.15199848583610853
293900000 0.18245717749453585
294000000 0.1431772334522009
294100000 0.16196704310801627
294200000 0.16606038618369234
294300000 0.1528677917991413
294400000 0.14077202322414517
best so far: 0
type: [1, 1, 1, 131, 1] 131
cases of this type: 2248091
294500000 0.08151365996580653
294600000 0.01135978405723638
294700000 0.010861203590273857
29480000